# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [240]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [241]:
# Calculate the total number of unique schools
totalschools = school_data_complete["school_name"].nunique()  
totalschools

15

In [242]:
# Calculate the total number of students
totalstudents = school_data_complete["Student ID"].nunique()
totalstudents

39170

In [243]:
# Calculate the total budget
totalbudget = school_data["budget"].sum()
totalbudget

24649428

In [244]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean() 
average_math_score

78.98537145774827

In [245]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [246]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(totalstudents) * 100
passing_math_percentage

74.9808526933878

In [247]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(totalstudents) * 100
passing_reading_percentage

85.80546336482001

In [248]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(totalstudents) * 100
overall_passing_rate

65.17232575950983

In [249]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary =  pd.DataFrame({
    "Total Schools" : [totalschools],
    "Total Students" : [totalstudents],
    "Total Budget" : [totalbudget],
    "Average Math Score" : [average_math_score],
    "Average Reading Score" : [average_reading_score],
    "Overall Passing Rate" : [overall_passing_rate],
    "% Passing Math": [passing_math_percentage],
    "% Passing Reading" : [passing_reading_percentage]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Passing Rate,% Passing Math,% Passing Reading
0,15,"39,170","$24,649,428.00",78.985371,81.87784,65.172326,74.980853,85.805463


## School Summary

In [250]:
# Use the code provided to select the school type
group_by_school=school_data_complete.set_index("school_name").groupby(["school_name"])

school_types = school_data.set_index(["school_name"])["type"]

In [251]:
# Calculate the total student count
per_school_counts = group_by_school["Student ID"].count()

In [252]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [253]:
# Calculate the average test scores
per_school_math = group_by_school["math_score"].mean()
per_school_reading = group_by_school["reading_score"].mean()

In [254]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name")["Student ID"].count()/per_school_counts

In [255]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name")["Student ID"].count()/per_school_counts

In [256]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)]


In [257]:
overall= (school_passing_math + school_passing_reading) / 2 * 100
print(overall)

school_name
Bailey High School       74.306672
Cabrera High School      95.586652
Figueroa High School     73.363852
Ford High School         73.804308
Griffin High School      95.265668
Hernandez High School    73.807983
Holden High School       94.379391
Huang High School        73.500171
Johnson High School      73.639992
Pena High School         95.270270
Rodriguez High School    73.293323
Shelton High School      94.860875
Thomas High School       95.290520
Wilson High School       95.203679
Wright High School       94.972222
Name: Student ID, dtype: float64


In [258]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": per_school_counts,
    "School Budget": per_school_budget,
    "Capita": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    '% Passing Math': school_passing_math,
    '% Passing Reading': school_passing_reading,
    "Overall Passing Rate": overall})

# Formatting
per_school_summary["School Budget"] = per_school_summary["School Budget"].map("${:,.2f}".format)
per_school_summary["Capita"] = per_school_summary["Capita"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary.style.format({'Total Students': '{:,}',
                          'Average Math Score': "{:.1f}", 
                          'Average Reading Score': "{:.1f}", 
                          "% Passing Math" : "{:.1%}", 
                          "% Passing Reading" : "{:.1%}", "Overall Passing Rate": "{:.1f}"})


,School Type,Total Students,School Budget,Capita,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.0,81.0,66.7%,81.9%,74.3
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84.0,94.1%,97.0%,95.6
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,66.0%,80.7%,73.4
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1,80.7,68.3%,79.3%,73.8
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93.4%,97.1%,95.3
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.3,80.9,66.8%,80.9%,73.8
Holden High School,Charter,427,"$248,087.00",$581.00,83.8,83.8,92.5%,96.3%,94.4
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6,81.2,65.7%,81.3%,73.5
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.1,81.0,66.1%,81.2%,73.6


## Highest-Performing Schools (by % Overall Passing)

In [259]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("Overall Passing Rate", ascending=False)

top_schools.head(5).style.format({'Total Students': '{:,}',
                           "Total School Budget": "${:,}", 
                           "Per Student Budget": "${:.0f}", 
                           "% Passing Math": "{:.1%}", 
                           "% Passing Reading": "{:.1%}", 
                           "Overall Passing Rate": "{:.1f}"})


,School Type,Total Students,School Budget,Capita,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.1%,97.0%,95.6
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.3%,97.3%,95.3
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.6%,95.9%,95.3
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.4%,97.1%,95.3
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.9%,96.5%,95.2


## Bottom Performing Schools (By % Overall Passing)

In [260]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = top_schools.tail()
bottom_schools = bottom_schools.sort_values("Overall Passing Rate")
bottom_schools.head(5).style.format({'Total Students': '{:,}', 
                       "Total School Budget": "${:,}", 
                       "Per Student Budget": "${:.0f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1f}"})

,School Type,Total Students,School Budget,Capita,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.4%,80.2%,73.3
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,66.0%,80.7%,73.4
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.7%,81.3%,73.5
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.1%,81.2%,73.6
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.3%,79.3%,73.8


## Math Scores by Grade

In [261]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]


# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({
        "9th": ninth_graders_scores,
        "10th": tenth_graders_scores,
        "11th": eleventh_graders_scores,
        "12th": twelfth_graders_scores
})
math_scores_by_grade = math_scores_by_grade[["9th","10th","11th","12th"]]

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


## Reading Score by Grade 

In [262]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
    "9th": ninth_graders_scores,
    "10th": tenth_graders_scores,
    "11th": eleventh_graders_scores,
    "12th": twelfth_graders_scores})


# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade.style.format({'9th': '{:.1f}', 
                             "10th": '{:.1f}', 
                             "11th": "{:.1f}", 
                             "12th": "{:.1f}"})

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


## Scores by School Spending

In [263]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels1 = ["<$585", "$585-630", "$630-645", "$645-680"]

In [264]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [265]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=labels1)
school_spending_df.head()

,School Type,Total Students,School Budget,Capita,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,0.666801,0.819333,74.306672,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,0.941335,0.970398,95.586652,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,0.659885,0.807392,73.363852,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,0.683096,0.792990,73.804308,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,0.933924,0.971390,95.265668,$585-630


In [266]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Overall Passing Rate"]

In [267]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
   "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending})

# Display results
spending_summary.style.format({"Average Math Score": "{:.1f}", "Average Reading Score":"{:.1f}","% Passing Math":"{:.1%}","% Passing Reading": "{:.1%}",
                               "% Overall Passing":"{:.1f}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.5,83.9,93.5%,96.6%,95.0
$585-630,81.9,83.2,87.1%,92.7%,89.9
$630-645,78.5,81.6,73.5%,84.4%,78.9
$645-680,77.0,81.0,66.2%,81.1%,73.6


## Scores by School Size

In [268]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [276]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=labels2)
per_school_summary

,School Type,Total Students,School Budget,Capita,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,School Size
school_name,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,0.666801,0.819333,74.306672,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,0.941335,0.970398,95.586652,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,0.659885,0.807392,73.363852,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,0.683096,0.792990,73.804308,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,0.933924,0.971390,95.265668,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,0.667530,0.808630,73.807983,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,0.925059,0.962529,94.379391,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,0.656839,0.813164,73.500171,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,0.660576,0.812224,73.639992,Large (2000-5000)


In [270]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["Overall Passing Rate"]

In [271]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing})

# Display results
size_summary.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.3%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8,83.9,93.6%,96.1%,94.824831
Medium (1000-2000),83.4,83.9,93.6%,96.8%,95.195187
Large (2000-5000),77.7,81.3,70.0%,82.8%,76.364998


## Scores by School Type

In [272]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary.groupby(["School Type"]).mean()["Overall Passing Rate"]



In [273]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score": type_math_scores,
    "Average Reading Score": type_reading_scores,
    "% Passing Math": type_passing_math,
    "% Passing Reading": type_passing_reading,
    "% Overall Passing": type_overall_passing})

# Display results
type_summary.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,93.6%,96.6%,95.103660
District,77.0,81.0,66.5%,80.8%,73.673757
